{/* cspell:ignore ccsd ffsim pvdz Qunova HiVQE AVAS Dooh pyscf nucmod GTO's lucj */}

In [ ]:
# This cell is hidden from users
# It stops the linter complaining about undefined variables
# ruff: noqa: F821

# HiVQE Chemistry - A Qiskit Function by Qunova Computing

<Admonition type="note">
  <LegacyContent>
This documentation is relevant to IBM Quantum&reg; Platform Classic. If you need the newer version, go to the new [IBM Quantum Platform documentation.](https://quantum.cloud.ibm.com/docs/guides/qunova-chemistry)
</LegacyContent>
<CloudContent>
This documentation is relevant to the new IBM Quantum&reg; Platform. If you need the previous version, return to the [IBM Quantum Platform Classic documentation.](https://docs.quantum.ibm.com/guides/qunova-chemistry)
</CloudContent>

  Qiskit Functions are an experimental feature available only to IBM Quantum&reg; Premium Plan users. They are in preview release status and subject to change.
</Admonition>

## Overview

In quantum chemistry, the electronic structure problem focuses on finding the solutions to the electronic Schrödinger equation - the quantum wave functions describing the behavior of the system's electrons. These wave functions are vectors of complex amplitudes, with each amplitude corresponding to the contribution of a possible electron configuration.

The ground state is the lowest energy wave function of the system and has a special importance in the study of molecular systems. The most accurate approach for computing the ground state considers all possible electron configurations, but this becomes intractable for larger systems since the number of configurations grows exponentially with system size.

The Handover Iterative Variational Quantum Eigensolver (HiVQE) is an innovative hybrid quantum-classical method for accurately estimating the ground state of molecular systems. It integrates quantum hardware with classical computing, using quantum processors to efficiently explore candidate electron configurations and calculating the resulting wave function on classical computers. By generating compact yet chemically accurate wave functions, HiVQE enhances research and discovery in quantum chemistry and materials science.

![Image showing an overview of Qunova's HiVQE Algorithm](/images/guides/qunova-chemistry/overview.avif)

HiVQE reduces the computational complexity of the electronic structure problem by efficiently estimating the ground state with high accuracy. It focuses on a carefully selected subset of the most relevant electron configurations, optimizing both accuracy and efficiency.

Combining the strengths of both classical and quantum computers, HiVQE iteratively refines and improves the current estimate wave function. Its unique subspace construction techniques help make configuration selection more efficient, so that users have greater computational control and improved accuracy in quantum chemistry simulations.

If you would like to learn about the algorithm in more depth, you can [read the associated research paper](https://qunova-my.sharepoint.com/:b:/g/personal/pellow_r_qunovacomputing_com/EcDyE4yQD_NAppzyUvhnlrABIMlqY3JhrLS41hkBPk22uQ?e=2dx3JT).

## Description

The number of electron configurations for a molecular system grows exponentially with system size. However, for certain electronic states, such as the ground state, it is common that only a small fraction of configurations significantly contribute to the state’s energy. Selected configuration interaction (SCI) methods exploit this sparsity to reduce computational costs by identifying and focusing on the most relevant configurations. This subset of configurations is referred to as a subspace.

HiVQE leverages the inherent efficiency of quantum computers for representing molecular systems to aid the subspace search. It integrates classical and quantum subroutines to solve the electronic structure problem with high accuracy. Unlike existing quantum SCI methods, HiVQE combines variational training, iterative subspace construction, and pre-diagonalization configuration screening to enhance efficiency by reducing quantum measurements, iterations, and classical diagonalization costs. HiVQE can therefore be applied to larger molecular systems which require more qubits, and reduces the cost to solve a problem of a given size to the same degree of accuracy.

![Image showing a detailed description of each step in Qunova's HiVQE algorithm.](/images/guides/qunova-chemistry/description.avif)

To compute a system's ground state, HiVQE first uses the classical chemistry package PySCF to generate a molecular representation from user-provided inputs, such as the molecular geometry and other molecular information. It then enters a hybrid quantum-classical optimization loop, iteratively refining a subspace to optimally represent the ground state while minimizing the number of configurations included. The loop continues until convergence criteria, such as subspace size or energy stability, are met, after which the computed ground state wave function and energy are output. These results can be used to construct accurate potential energy surfaces and perform further analysis of the system.

The optimization loop focuses on adjusting a quantum circuit’s parameters to generate a high-quality subspace. HiVQE offers three quantum circuit options: [ExcitationPreserving](/api/qiskit/qiskit.circuit.library.ExcitationPreserving#excitationpreserving), [EfficientSU2](/api/qiskit/qiskit.circuit.library.EfficientSU2#efficientsu2), and [LUCJ](https://qiskit-community.github.io/ffsim/dev/explanations/lucj.html#The-local-unitary-cluster-Jastrow-(LUCJ)-ansatz). The optimization is initialized close to the Hartree-Fock reference state due to its general suitability. The circuit is then executed on a quantum device and configurations are sampled from the resulting quantum state before being returned as binary strings. Due to quantum device noise, some sampled configurations may be physically invalid, failing to conserve electron number or spin. HiVQE addresses this using the configuration recovery process from the [qiskit-addon-sqd](/guides/qiskit-addons-sqd#sample-based-quantum-diagonalization-sqd-overview) package, so that users can either correct invalid configurations or discard them.

The valid configurations then undergo an optional screening step to remove those predicted to contribute minimally. This reduces the dimension of the subspace, thereby lowering the cost of the diagonalization step. If the screening is enabled, then a preliminary subspace Hamiltonian is constructed from the valid configurations and a diagonalization is performed with very loose termination critieria. Though the accuracy of the resulting amplitudes for each configuration is low, it is effective for predicting which configurations to leave out of the subspace this iteration, and it is fast to compute.

The selected configurations are added to the subspace, and the system's Hamiltonian is projected into this subspace. The subspace updates iteratively, preserving the most relevant configurations across iterations. This approach contrasts with alternative methods because the quantum circuit does not need to approximate the full ground state at each step.

Next, the subspace Hamiltonian is classically diagonalized to obtain the lowest eigenvalue and its corresponding eigenvector, representing an approximation of the ground state and its energy. As the subspace quality improves through iterations, the computed ground state better approximates the true ground state. An additional screening step can be performed at this point to remove any configurations from the subspace that don't have a substantial contribution to the computed ground state. This step ensures that the subspace carried into the next iteration is as compact as possible. This is evaluated based on the amplitudes that are returned by the diagonalization, as these represent each configuration's importance contribution to the computed ground state.

A convergence check then determines whether further training would improve results. If so, then an optional classical expansion step is performed, the quantum circuit parameters are updated to further minimize the computed energy, and the process repeats. The classical expansion step generates additional configurations for the subspace, supplementing the configurations sampled from the quantum device. It first identifies the configuration with the largest amplitude in the diagonalization results, before generating new configurations with single and double excitations from the identified configuration. The desired number of these configurations are then added to the subspace.

Once it is determined that the iterations have converged, HiVQE returns the computed ground state (in the form of the states in the subspace and their amplitudes in the ground state wave function), its energy, and an energy variance measure that gives an indication of whether the computed state forms an eigenstate of the system's Hamiltonian.

Users are able to decide the quantum circuit used and the number of shots taken for each quantum circuit, as well as control the subspace size or enable the classical generation of additional configurations to assist the quantum generated configurations. In this way users can tailor the behavior of HiVQE to suit their desired applications.

## Get started

First, [request access to the function](https://forms.office.com/r/zN3hvMTqJ1).
<LegacyContent>
Then, authenticate using your [IBM Quantum&reg; API key](http://quantum.ibm.com/) and select the Qiskit Function as follows:
</LegacyContent>
<CloudContent>
Then, authenticate using your [IBM Quantum&reg; API key](http://quantum.cloud.ibm.com/) and select the Qiskit Function as follows:
</CloudContent>

In [ ]:
from qiskit_ibm_catalog import QiskitFunctionsCatalog

catalog = QiskitFunctionsCatalog()
function = catalog.load("qunova/hivqe-chemistry")

## Inputs

See the following table for all input parameters the function accepts. The subsequent [Molecule options](#molecule-options) and [HiVQE options](#hivqe-options) sections go into more details about those arguments.

<LegacyContent>
| Name                   | Type                                                           | Description                                                                                                                                                                                                                                                                                                 | Required | Default                                                                  | Example                                                                                   |
|------------------------|----------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------|--------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| geometry               | Union[List[List[Union[str, Tuple[float, float, float]]]], str] | This can be either a string or structured lists containing atom and coordinate pairs. If this is given as a string, it must be a molecule geometry in Cartesian Coordinate Format. If this is given as a list, it should be given as a list of lists that each contain an atom string and coordinate tuple. | Yes      | N/A                                                                      | `[['O', (0, 0, 0)], ['H', (0, 1, 0)], ['H', (0, 0, 1)]]` or `"O 0 0 0; H 0 1 0; H 0 0 1"` |
| backend\_name          | str                                                            | Name of the backend to make the query.                                                                                                                                                                                                                                                                      | Yes      | N/A                                                                      | `ibm_fez`                                                                                 |
| instance               | str                                                            | The instance to use in that format.                                                                                                                                                                                                                                                                | No       | One is randomly picked if your account has access to multiple instances. | `hub1/group1/project1`                                                                    |
| max\_states            | int                                                            | The maximum subspace dimension for the diagonalization. Fewer states will be used if the number is not a perfect square.                                                                                                                                                                                                                                                    | Yes      | N/A                                                                      | `100`                                                                                     |
| max\_expansion\_states | int                                                            | The maximum number of classically-generated CI states to be included in each iteration.                                                                                                                                                                                                                     | Yes      | N/A                                                                      | `10`                                                                                      |
| molecule_options                | dict                                                           | Options related to the molecule used as input to HiVQE. See the [Molecule options](#molecule-options) section for more details.                                                                                                                                                                                                                                                | No       | See the [Molecule options](#molecule-options) section for details.                                 | `{"basis": "sto3g", "unit": "angstrom" }`                               |
| hivqe_options                | dict                                                           | Options controlling the behavior of the HiVQE algorithm. See the [HiVQE options](#hivqe-options) section for more details.                                                                                                                                                                                                                                                | No       | See the [HiVQE options](#hivqe-options) section for details.                                 | `{"shots": 10_000, "max_iter": 10 }`                               |
</LegacyContent>
<CloudContent>
| Name                   | Type                                                           | Description                                                                                                                                                                                                                                                                                                 | Required | Default                                                                  | Example                                                                                   |
|------------------------|----------------------------------------------------------------|-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|----------|--------------------------------------------------------------------------|-------------------------------------------------------------------------------------------|
| geometry               | Union[List[List[Union[str, Tuple[float, float, float]]]], str] | This can be either a string or structured lists containing atom and coordinate pairs. If this is given as a string, it must be a molecule geometry in Cartesian Coordinate Format. If this is given as a list, it should be given as a list of lists that each contain an atom string and coordinate tuple. | Yes      | N/A                                                                      | `[['O', (0, 0, 0)], ['H', (0, 1, 0)], ['H', (0, 0, 1)]]` or `"O 0 0 0; H 0 1 0; H 0 0 1"` |
| backend\_name          | str                                                            | Name of the backend to make the query.                                                                                                                                                                                                                                                                      | Yes      | N/A                                                                      | `ibm_fez`                                                                                 |
| instance               | str                                                            | The instance to use in that format.                                                                                                                                                                                                                                                                | No       | One is randomly picked if your account has access to multiple instances. | `CRN`                                                                    |
| max\_states            | int                                                            | The maximum subspace dimension for the diagonalization. Fewer states will be used if the number is not a perfect square.                                                                                                                                                                                                                                                    | Yes      | N/A                                                                      | `100`                                                                                     |
| max\_expansion\_states | int                                                            | The maximum number of classically-generated CI states to be included in each iteration.                                                                                                                                                                                                                     | Yes      | N/A                                                                      | `10`                                                                                      |
| molecule_options                | dict                                                           | Options related to the molecule used as input to HiVQE. See the [Molecule options](#molecule-options) section for more details.                                                                                                                                                                                                                                                | No       | See the [Molecule options](#molecule-options) section for details.                                 | `{"basis": "sto3g", "unit": "angstrom" }`                               |
| hivqe_options                | dict                                                           | Options controlling the behavior of the HiVQE algorithm. See the [HiVQE options](#hivqe-options) section for more details.                                                                                                                                                                                                                                                | No       | See the [HiVQE options](#hivqe-options) section for details.                                 | `{"shots": 10_000, "max_iter": 10 }`                               |
</CloudContent>

### Molecule options

The following table details all keys and values that can be set in the `molecule_options` dictionary, as well as their data types and default values. All keys are optional.

| Key                               | Value type                          | Default Value                    | Valid range                                                                                                                                                    | Explanation                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|:----------------------------------|:-----------------------------------:|:--------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `"charge"`                        | `int`                               | `0`                              | Various                                                                                                                                                        | An integer specifying the total net charge of the molecular system. The default value is 0; however, it can be any integer.                                                                                                                                                                                                                                                                                                                                                                                                              |
| `"basis"`                         | `str`                               | `'sto-3g'`                       | Various                                                                                                                                                        | A string specifying the basis type; these are passed to pyscf. (eg: `"sto-3g"`, `"3-21g"`, `"6-31g"`, `"cc-pvdz"`)                                                                                                                                                                                                                                                                                                                                                                                                                      |
| `"active_orbitals"`               | `List[int]`                         | Every orbital index.             | The spatial orbital indices valid for the problem.                                                                                                             | A list of active orbital indices in the interval [0, n) where n is the number of qubits used in the problem. If this is specified, then the frozen_orbitals argument must also be specified.                                                                                                                                                                                                                                                                                                                                            |
| `"frozen_orbitals"`               | `List[int]`                         | No indices.                      | The spatial orbital indices valid for the problem, excluding active orbitals.                                                                                  | A list of frozen orbital indices in the same range as active_orbitals. If specified, then active_orbitals must also be specified. Note that only occupied orbitals should be frozen as the number of active electrons is reduced by 2 for every occupied orbital that is frozen.                                                                                                                                                                                                                                                        |
| `"orbital_coeffs"`                | `List[List[float]]`                 | Hartree-Fock molecular orbitals. | Various.                                                                                                                                                       | The coefficients for the spatial orbitals used in the calculation of the electronic repulsion integrals for the system. Some valid examples are Hartree-Fock molecular orbitals, natural orbitals, and AVAS orbitals.                                                                                                                                                                                                                                                                                                                   |
| `"symmetry"`                      | `Union[str, bool]`                  | `False`                          | `True` or `False`                                                                                                                                              | Used to invoke point group symmetry for the initial molecular calculations to construct the symmetry adapted orbital basis. These symmetry-adapted orbitals are used as basis functions for the following SCF calculations. The default value is False; if set to True, then it will be invoked and arbitrary point groups will automatically be detected and used. If a particular symmetry is assigned, for example, symmetry = “Dooh”, then an error will be raised if the molecular geometry is not subject to this required symmetry. |
| `"symmetry_subgroup"`             | `Optional[str]`                     | `None`                           | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Can be used to generate a subgroup of the detected symmetry. This has no effect when symmetry is specified using the symmetry keyword argument.                                                                                                                                                                                                                                                                                                                                                                                         |
| `"unit"`                          | `str`                               | `"angstrom"`                     | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Specifies the measurement unit to use for atomic coordinates and distances. The default is to use angstrom units.                                                                                                                                                                                                                                                                                                                                                                                                                       |
| `"nucmod"`                        | `Optional[Union[dict, str]]`        | `None`                           | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Specifies the nuclear model to be used. By default it uses the point nuclear model, and other values enable Gaussian nuclear model. If a function is given, it will be used with the Gaussian nuclear model to generate the nuclear charge distribution value 'zeta'.                                                                                                                                                                                                                                                                   |
| `"pseudo"`                        | `Optional[Union[dict, str]]`        | `None`                           | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Specifies the pseudopotential for the atoms in the molecule. This is None by default, indicating that no pseudopotentials are applied and all electrons are explicitly included in the calculations.                                                                                                                                                                                                                                                                                                                                                |
| `"cart"`                          | `bool`                              | `False`                          | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Specifies whether to use cartesian GTOs as the angular momentum basis functions in the calculation. The default value of False will use spherical GTOs.                                                                                                                                                                                                                                                                                                                                                                               |
| `"magmom"`                        | `Optional[List[Union[int, float]]]` | `None`                           | See [pyscf documentation](https://pyscf.org/pyscf_api_docs/pyscf.gto.html#pyscf.gto.mole.MoleBase.build).                                                      | Sets the colinear spin magnetic moment of each atom. By default, this is None and each atom is initialized with a spin of zero.                                                                                                                                                                                                                                                                                                                                                                                                         |
| `"avas_aolabels"`                 | `Optional[List[str]]`               | `None`                           | i.e. ["H 1s", "O 2p"] for H2O                                                                                                                                                             | This defines Atomic Orbital to be included in AVAS scheme. See [AVAS documentation](https://arxiv.org/abs/1701.07862) .                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| `"avas_threshold"`                | `float`                             | `0.2`                            |  Between 0.0 and 2.0                                                                                                                                                      |  This specifies the cutoff value used to determine which Atomic Orbitals (AOs) are retained in the active space.                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| `"noons_level"`                   | `Optional[str]`                     | `None`                           | `"mp2"` or `"ccsd"`                                                                                                                                            | This defines the theoretical approach for preparing natural orbitals and selecting active orbitals based on the Natural Orbital Occupation Numbers (NOONs) scheme. See [NOONs documentation](https://doi.org/10.1063/5.0042147). Both the active and frozen orbital indices must be provided to control the number of orbitals (and the number of qubits).                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |

### HiVQE options

The following table details all keys and values that can be set in the `hivqe_options` dictionary, as well as their data types and default values. All keys are optional.

| Key                               | Value type                          | Default Value                    | Valid range                                                                                                                                                    | Explanation                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            |
|:----------------------------------|:-----------------------------------:|:--------------------------------:|:---------------------------------------------------------------------------------------------------------------------------------------------------------------|:----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| `"shots"`                         | `int`                               | `1_000`                          | Between 1 and 10 000.                                                                                                                                          | Number of shots to use on the quantum device per iteration.                                                                                                                                                                                                                                                                                                                                                                                                                                                                             |
| `"max_iter"`                      | `int`                               | `25`                             | Between 1 and 50.                                                                                                                                              | The maximum number of iterations to run to optimize the ansatz. The algorithm may use fewer iterations if convergence is achieved early.                                                                                                                                                                                                                                                                                                                                                                                                 |
| `"initial_basis_states"`          | `List[str]`                         | The Hartree-Fock state.          | Binary strings with the number of bits matching the required number of qubits for the problem.                                                                 | Can be used to restart the algorithm with classical states from a previous result.                                                                                                                                                                                                                                                                                                                                                                                                                                                      |
| `"ansatz"`                        | `str`                               | `"epa"`                          | `"epa"`, `"hea"`, or `"lucj"`                                                                                                                                  | This specifies the quantum ansatz to optimize to generate new states. `"epa"` selects the excitation-preserving ansatz. `"hea"` selects the hardware-efficient ansatz. `"lucj"` selects the local unitary cluster Jastrow ansatz.                                                                                                                                                                                                                                                                                                       |
| `"convergence_count"`             | `int`                               | `3`                              | At least 2.                                                                                                                                                    | The number of iterations without significant change of the computed energy that should elapse before the algorithm is deemed to have converged.                                                                                                                                                                                                                                                                                                                                                                                         |
| `"convergence_abstol"`            | `float`                             | `1e-4`                           | More than 0 and at most 1.                                                                                                                                     | The magnitude of change in computed energy that is considered significant for the purposes of convergence checks.                                                                                                                                                                                                                                                                                                                                                                                                                       |
| `"reset_convergence_count"`       | `bool`                              | `True`                           | `True` or `False`                                                                                                                                              | If `True`, the `convergence_count` iterations must occur without an interrupting significant change to qualify as converging. If `False`, then the algorithm will stop after `convergence_count` if insignificant changes have occurred at any iterations during the optimization process.                                                                                                                                                                                                                                                 |
| `"configuration_recovery"`        | `bool`                              | `True`                           | `True` or `False`.                                                                                                                                             | Whether or not to use configuration recovery from the `qiskit-addon-sqd` package. If True, invalid states sampled from the quantum device are corrected classically. If False, they are discarded.                                                                                                                                                                                                                                                                                                                                      |
| `"ansatz_entanglement"`           | `str`                               | `"circular"`                     | Any one of `"linear"`, `"reverse_linear"`, `"pairwise"`, `"circular"`, `"full"`, or `"sca"`. If using the `"lucj"` ansatz, `"lucj_default"` is also an option. | This specifies the entanglement scheme that should be used within the quantum circuit, following common Qiskit and [ffsim conventions for the LUCJ ansatz](https://qiskit-community.github.io/ffsim/dev/how-to-guides/lucj.html#How-to-simulate-the-local-unitary-cluster-Jastrow-(LUCJ)-ansatz).                                                                                                                                                                                                                                       |
| `"ansatz_reps"`                   | `int`                               | `2`                              | Greater than 0.                                                                                                                                                | The number of repetitions of each layer in the quantum circuit.                                                                                                                                                                                                                                                                                                                                                                                                                                                                         |
| `"amplitude_screening_tolerance"` | `Union[float,int]`                  | `0`                              | At least 0, and less than 1.                                                                                                                                   | The tolerance for deciding which states should be screened out of the subspace after diagonalization. It specifies the inclusion threshold for the subspace states based on their computed amplitudes.                                                                                                                                                                                                                                                                                                                                  |
| `"overlap_screening_tolerance"`   | `float`                             | `1e-2`                           | Between `1e-4` and `1e-1`, inclusive.                                                                                                                          | The tolerance for predicting which states should be screened out of the subspace prior to diagonalization. It controls the accuracy of the predicted amplitudes for each state, with a smaller value resulting in more accurate predictions.                                                                                                                                                                                                                                                                                            |

## Outputs

The function returns a dictionary with four keys and values. The keys and values are documented in the following table:

| Key             | Value Type    | Explanation                                                                                                               |
|:----------------|---------------|---------------------------------------------------------------------------------------------------------------------------|
| `"energy"`      | `float`       | The approximate ground state energy of the molecule.                                                                      |
| `"states"`      | `List[str]`   | The selected determinants that form the subspace used to solve for the energy. They are in alternating alpha-beta format. |
| `"eigenvector"` | `List[float]` | The eigenvector corresponding to the ground state of the subspace composed of `"states"`.                                 |
| `"energy_variance"` | `float` | The energy variance of the ground state of the subspace composed of `"states"`, giving an indication of the quality of the solution. This value is non-negative and a lower value means that the ground state of the subspace more closely approximates an eigenstate of the system's Hamiltonian. |
| `"energy_history"` | `List[float]` | The energies computed each iteration during the hybrid optimization process, in the same order that they were computed. Two energies are computed per iteration as part of the SPSA optimization process. |

## Example

The first example shows how to compute the ground state energy for an NH3 molecule using the HiVQE algorithm.

#### Define the molecular geometry and options

The molecular geometry of NH3 is provided with Cartesian coordinates separated with ";" for each atom.

In [ ]:
# Define the molecule geometry
geometry = """
N         -0.85188       -0.02741        0.03141;
H          0.16545        0.00593       -0.01648;
H         -1.16348       -0.39357       -0.86702;
H         -1.16348        0.94228        0.06281;
"""

Additional options can be defined and provided for molecular system in the following dictionary format.

In [ ]:
# Configure some options for the job.
molecule_options = {"basis": "sto3g"}
hivqe_options = {"shots": 100, "max_iter": 20}

Execute the function with geometry and option inputs.

In [ ]:
# Run HiVQE
job = function.run(
    geometry=geometry,
    backend_name=backend_name,  # e.g. "ibm_fez"
    instance=instance,  # e.g. "ibm-q/open/main"
    max_states=2000,
    max_expansion_states=10,
    molecule_options=molecule_options,
    hivqe_options=hivqe_options,
)

It is a good idea to print the Function job ID so that it can be provided in support requests if something goes wrong.

In [ ]:
print("Job ID:", job.job_id)

This example then utilizes 16 qubits with 8 orbitals of sto3g basis for an NH3 molecule.

Check your Qiskit Function workload's [status](/guides/functions#check-job-status) or return [results](/guides/functions#retrieve-results) as follows:

In [ ]:
print(job.status())

After the job is completed, the results can be obtained with `result()` instance.

In [ ]:
result = job.result()
print(result)

Output:

In [ ]:
{
    "energy": np.float64(-55.521064587741876),
    "eigenvector": array(
        [
            9.82446681e-01,
            9.53706109e-03,
            3.09707825e-06,
            ...,
            -1.79658131e-07,
            3.41576228e-09,
            1.58456937e-03,
        ]
    ),
    "states": [
        "1111111111000000",
        "1111111110010000",
        "1111111011010000",
        "1111101111010000",
        "1110111111010000",
        "1011111111010000",
        "1111111110000100",
        "1111111011000100",
        "1111101111000100",
        "1110111111000100",
        "1011111111000100",
        "1111111010010100",
        "1111101110010100",
        "1110111110010100",
        "1011111110010100",
        "1111101011010100",
        "1110111011010100",
        "1011111011010100",
        "1110101111010100",
        "1111111110000001",
        "1111111011000001",
        "1111101111000001",
        "1110111111000001",
        "1011111111000001",
        "1111111010010001",
        "1111101110010001",
        "1110111110010001",
        "1011111110010001",
        "1111101011010001",
        "1110111011010001",
        "1011111011010001",
        "1110101111010001",
        "1111111010000101",
        "1111101110000101",
        "1110111110000101",
        "1011111110000101",
        "1111101011000101",
        "1110111011000101",
        "1011111011000101",
        "1110101111000101",
        "1111111101100000",
        "1111111100110000",
        "1111111001110000",
        "1111101101110000",
        "1110111101110000",
        "1011111101110000",
        "1111111100100100",
        "1111111001100100",
        "1111101101100100",
        "1110111101100100",
        "1011111101100100",
        "1111111000110100",
        "1111101100110100",
        "1110111100110100",
        "1011111100110100",
        "1111101001110100",
        "1110111001110100",
        "1011111001110100",
        "1110101101110100",
        "1111111100100001",
        "1111111001100001",
        "1111101101100001",
        "1110111101100001",
        "1011111101100001",
        "1111111000110001",
        "1111101100110001",
        "1110111100110001",
        "1011111100110001",
        "1111101001110001",
        "1110111001110001",
        "1011111001110001",
        "1110101101110001",
        "1111111000100101",
        "1111101100100101",
        "1110111100100101",
        "1011111100100101",
        "1111101001100101",
        "1110111001100101",
        "1011111001100101",
        "1110101101100101",
        "1111110111100000",
        "1111110110110000",
        "1111110011110000",
        "1111100111110000",
        "1110110111110000",
        "1011110111110000",
        "1111110110100100",
        "1111110011100100",
        "1111100111100100",
        "1110110111100100",
        "1011110111100100",
        "1111110010110100",
        "1111100110110100",
        "1110110110110100",
        "1011110110110100",
        "1111100011110100",
        "1110110011110100",
        "1011110011110100",
        "1110100111110100",
        "1111110110100001",
        "1111110011100001",
        "1111100111100001",
        "1110110111100001",
        "1011110111100001",
        "1111110010110001",
        "1111100110110001",
        "1110110110110001",
        "1011110110110001",
        "1111100011110001",
        "1110110011110001",
        "1011110011110001",
        "1110100111110001",
        "1111110010100101",
        "1111100110100101",
        "1110110110100101",
        "1011110110100101",
        "1111100011100101",
        "1110110011100101",
        "1011110011100101",
        "1110100111100101",
        "1111011111100000",
        "1111011110110000",
        "1111011011110000",
        "1111001111110000",
        "1110011111110000",
        "1011011111110000",
        "1111011110100100",
        "1111011011100100",
        "1111001111100100",
        "1110011111100100",
        "1011011111100100",
        "1111011010110100",
        "1111001110110100",
        "1110011110110100",
        "1011011110110100",
        "1111001011110100",
        "1110011011110100",
        "1011011011110100",
        "1110001111110100",
        "1111011110100001",
        "1111011011100001",
        "1111001111100001",
        "1110011111100001",
        "1011011111100001",
        "1111011010110001",
        "1111001110110001",
        "1110011110110001",
        "1011011110110001",
        "1111001011110001",
        "1110011011110001",
        "1011011011110001",
        "1110001111110001",
        "1111011010100101",
        "1111001110100101",
        "1110011110100101",
        "1011011110100101",
        "1111001011100101",
        "1110011011100101",
        "1011011011100101",
        "1110001111100101",
        "1101111111100000",
        "1101111110110000",
        "1101111011110000",
        "1101101111110000",
        "1100111111110000",
        "1001111111110000",
        "1101111110100100",
        "1101111011100100",
        "1101101111100100",
        "1100111111100100",
        "1001111111100100",
        "1101111010110100",
        "1101101110110100",
        "1100111110110100",
        "1001111110110100",
        "1101101011110100",
        "1100111011110100",
        "1001111011110100",
        "1100101111110100",
        "1101111110100001",
        "1101111011100001",
        "1101101111100001",
        "1100111111100001",
        "1001111111100001",
        "1101111010110001",
        "1101101110110001",
        "1100111110110001",
        "1001111110110001",
        "1101101011110001",
        "1100111011110001",
        "1001111011110001",
        "1100101111110001",
        "1101111010100101",
        "1101101110100101",
        "1100111110100101",
        "1001111110100101",
        "1101101011100101",
        "1100111011100101",
        "1001111011100101",
        "1100101111100101",
        "0111111111100000",
        "0111111110110000",
        "0111111011110000",
        "0111101111110000",
        "0110111111110000",
        "0011111111110000",
        "0111111110100100",
        "0111111011100100",
        "0111101111100100",
        "0110111111100100",
        "0011111111100100",
        "0111111010110100",
        "0111101110110100",
        "0110111110110100",
        "0011111110110100",
        "0111101011110100",
        "0110111011110100",
        "0011111011110100",
        "0110101111110100",
        "0111111110100001",
        "0111111011100001",
        "0111101111100001",
        "0110111111100001",
        "0011111111100001",
        "0111111010110001",
        "0111101110110001",
        "0110111110110001",
        "0011111110110001",
        "0111101011110001",
        "0110111011110001",
        "0011111011110001",
        "0110101111110001",
        "0111111010100101",
        "0111101110100101",
        "0110111110100101",
        "0011111110100101",
        "0111101011100101",
        "0110111011100101",
        "0011111011100101",
        "0110101111100101",
        "1111111101001000",
        "1111111100011000",
        "1111111001011000",
        "1111101101011000",
        "1110111101011000",
        "1011111101011000",
        "1111111100001100",
        "1111111001001100",
        "1111101101001100",
        "1110111101001100",
        "1011111101001100",
        "1111111000011100",
        "1111101100011100",
        "1110111100011100",
        "1011111100011100",
        "1111101001011100",
        "1110111001011100",
        "1011111001011100",
        "1110101101011100",
        "1111111100001001",
        "1111111001001001",
        "1111101101001001",
        "1110111101001001",
        "1011111101001001",
        "1111111000011001",
        "1111101100011001",
        "1110111100011001",
        "1011111100011001",
        "1111101001011001",
        "1110111001011001",
        "1011111001011001",
        "1110101101011001",
        "1111111000001101",
        "1111101100001101",
        "1110111100001101",
        "1011111100001101",
        "1111101001001101",
        "1110111001001101",
        "1011111001001101",
        "1110101101001101",
        "1111110111001000",
        "1111110110011000",
        "1111110011011000",
        "1111100111011000",
        "1110110111011000",
        "1011110111011000",
        "1111110110001100",
        "1111110011001100",
        "1111100111001100",
        "1110110111001100",
        "1011110111001100",
        "1111110010011100",
        "1111100110011100",
        "1110110110011100",
        "1011110110011100",
        "1111100011011100",
        "1110110011011100",
        "1011110011011100",
        "1110100111011100",
        "1111110110001001",
        "1111110011001001",
        "1111100111001001",
        "1110110111001001",
        "1011110111001001",
        "1111110010011001",
        "1111100110011001",
        "1110110110011001",
        "1011110110011001",
        "1111100011011001",
        "1110110011011001",
        "1011110011011001",
        "1110100111011001",
        "1111110010001101",
        "1111100110001101",
        "1110110110001101",
        "1011110110001101",
        "1111100011001101",
        "1110110011001101",
        "1011110011001101",
        "1110100111001101",
        "1111011111001000",
        "1111011110011000",
        "1111011011011000",
        "1111001111011000",
        "1110011111011000",
        "1011011111011000",
        "1111011110001100",
        "1111011011001100",
        "1111001111001100",
        "1110011111001100",
        "1011011111001100",
        "1111011010011100",
        "1111001110011100",
        "1110011110011100",
        "1011011110011100",
        "1111001011011100",
        "1110011011011100",
        "1011011011011100",
        "1110001111011100",
        "1111011110001001",
        "1111011011001001",
        "1111001111001001",
        "1110011111001001",
        "1011011111001001",
        "1111011010011001",
        "1111001110011001",
        "1110011110011001",
        "1011011110011001",
        "1111001011011001",
        "1110011011011001",
        "1011011011011001",
        "1110001111011001",
        "1111011010001101",
        "1111001110001101",
        "1110011110001101",
        "1011011110001101",
        "1111001011001101",
        "1110011011001101",
        "1011011011001101",
        "1110001111001101",
        "1101111111001000",
        "1101111110011000",
        "1101111011011000",
        "1101101111011000",
        "1100111111011000",
        "1001111111011000",
        "1101111110001100",
        "1101111011001100",
        "1101101111001100",
        "1100111111001100",
        "1001111111001100",
        "1101111010011100",
        "1101101110011100",
        "1100111110011100",
        "1001111110011100",
        "1101101011011100",
        "1100111011011100",
        "1001111011011100",
        "1100101111011100",
        "1101111110001001",
        "1101111011001001",
        "1101101111001001",
        "1100111111001001",
        "1001111111001001",
        "1101111010011001",
        "1101101110011001",
        "1100111110011001",
        "1001111110011001",
        "1101101011011001",
        "1100111011011001",
        "1001111011011001",
        "1100101111011001",
        "1101111010001101",
        "1101101110001101",
        "1100111110001101",
        "1001111110001101",
        "1101101011001101",
        "1100111011001101",
        "1001111011001101",
        "1100101111001101",
        "0111111111001000",
        "0111111110011000",
        "0111111011011000",
        "0111101111011000",
        "0110111111011000",
        "0011111111011000",
        "0111111110001100",
        "0111111011001100",
        "0111101111001100",
        "0110111111001100",
        "0011111111001100",
        "0111111010011100",
        "0111101110011100",
        "0110111110011100",
        "0011111110011100",
        "0111101011011100",
        "0110111011011100",
        "0011111011011100",
        "0110101111011100",
        "0111111110001001",
        "0111111011001001",
        "0111101111001001",
        "0110111111001001",
        "0011111111001001",
        "0111111010011001",
        "0111101110011001",
        "0110111110011001",
        "0011111110011001",
        "0111101011011001",
        "0110111011011001",
        "0011111011011001",
        "0110101111011001",
        "0111111010001101",
        "0111101110001101",
        "0110111110001101",
        "0011111110001101",
        "0111101011001101",
        "0110111011001101",
        "0011111011001101",
        "0110101111001101",
        "1111110101101000",
        "1111110100111000",
        "1111110001111000",
        "1111100101111000",
        "1110110101111000",
        "1011110101111000",
        "1111110100101100",
        "1111110001101100",
        "1111100101101100",
        "1110110101101100",
        "1011110101101100",
        "1111110000111100",
        "1111100100111100",
        "1110110100111100",
        "1011110100111100",
        "1111100001111100",
        "1110110001111100",
        "1011110001111100",
        "1110100101111100",
        "1111110100101001",
        "1111110001101001",
        "1111100101101001",
        "1110110101101001",
        "1011110101101001",
        "1111110000111001",
        "1111100100111001",
        "1110110100111001",
        "1011110100111001",
        "1111100001111001",
        "1110110001111001",
        "1011110001111001",
        "1110100101111001",
        "1111110000101101",
        "1111100100101101",
        "1110110100101101",
        "1011110100101101",
        "1111100001101101",
        "1110110001101101",
        "1011110001101101",
        "1110100101101101",
        "1111011101101000",
        "1111011100111000",
        "1111011001111000",
        "1111001101111000",
        "1110011101111000",
        "1011011101111000",
        "1111011100101100",
        "1111011001101100",
        "1111001101101100",
        "1110011101101100",
        "1011011101101100",
        "1111011000111100",
        "1111001100111100",
        "1110011100111100",
        "1011011100111100",
        "1111001001111100",
        "1110011001111100",
        "1011011001111100",
        "1110001101111100",
        "1111011100101001",
        "1111011001101001",
        "1111001101101001",
        "1110011101101001",
        "1011011101101001",
        "1111011000111001",
        "1111001100111001",
        "1110011100111001",
        "1011011100111001",
        "1111001001111001",
        "1110011001111001",
        "1011011001111001",
        "1110001101111001",
        "1111011000101101",
        "1111001100101101",
        "1110011100101101",
        "1011011100101101",
        "1111001001101101",
        "1110011001101101",
        "1011011001101101",
        "1110001101101101",
        "1101111101101000",
        "1101111100111000",
        "1101111001111000",
        "1101101101111000",
        "1100111101111000",
        "1001111101111000",
        "1101111100101100",
        "1101111001101100",
        "1101101101101100",
        "1100111101101100",
        "1001111101101100",
        "1101111000111100",
        "1101101100111100",
        "1100111100111100",
        "1001111100111100",
        "1101101001111100",
        "1100111001111100",
        "1001111001111100",
        "1100101101111100",
        "1101111100101001",
        "1101111001101001",
        "1101101101101001",
        "1100111101101001",
        "1001111101101001",
        "1101111000111001",
        "1101101100111001",
        "1100111100111001",
        "1001111100111001",
        "1101101001111001",
        "1100111001111001",
        "1001111001111001",
        "1100101101111001",
        "1101111000101101",
        "1101101100101101",
        "1100111100101101",
        "1001111100101101",
        "1101101001101101",
        "1100111001101101",
        "1001111001101101",
        "1100101101101101",
        "0111111101101000",
        "0111111100111000",
        "0111111001111000",
        "0111101101111000",
        "0110111101111000",
        "0011111101111000",
        "0111111100101100",
        "0111111001101100",
        "0111101101101100",
        "0110111101101100",
        "0011111101101100",
        "0111111000111100",
        "0111101100111100",
        "0110111100111100",
        "0011111100111100",
        "0111101001111100",
        "0110111001111100",
        "0011111001111100",
        "0110101101111100",
        "0111111100101001",
        "0111111001101001",
        "0111101101101001",
        "0110111101101001",
        "0011111101101001",
        "0111111000111001",
        "0111101100111001",
        "0110111100111001",
        "0011111100111001",
        "0111101001111001",
        "0110111001111001",
        "0011111001111001",
        "0110101101111001",
        "0111111000101101",
        "0111101100101101",
        "0110111100101101",
        "0011111100101101",
        "0111101001101101",
        "0110111001101101",
        "0011111001101101",
        "0110101101101101",
        "1111010111101000",
        "1111010110111000",
        "1111010011111000",
        "1111000111111000",
        "1110010111111000",
        "1011010111111000",
        "1111010110101100",
        "1111010011101100",
        "1111000111101100",
        "1110010111101100",
        "1011010111101100",
        "1111010010111100",
        "1111000110111100",
        "1110010110111100",
        "1011010110111100",
        "1111000011111100",
        "1110010011111100",
        "1011010011111100",
        "1110000111111100",
        "1111010110101001",
        "1111010011101001",
        "1111000111101001",
        "1110010111101001",
        "1011010111101001",
        "1111010010111001",
        "1111000110111001",
        "1110010110111001",
        "1011010110111001",
        "1111000011111001",
        "1110010011111001",
        "1011010011111001",
        "1110000111111001",
        "1111010010101101",
        "1111000110101101",
        "1110010110101101",
        "1011010110101101",
        "1111000011101101",
        "1110010011101101",
        "1011010011101101",
        "1110000111101101",
        "1101110111101000",
        "1101110110111000",
        "1101110011111000",
        "1101100111111000",
        "1100110111111000",
        "1001110111111000",
        "1101110110101100",
        "1101110011101100",
        "1101100111101100",
        "1100110111101100",
        "1001110111101100",
        "1101110010111100",
        "1101100110111100",
        "1100110110111100",
        "1001110110111100",
        "1101100011111100",
        "1100110011111100",
        "1001110011111100",
        "1100100111111100",
        "1101110110101001",
        "1101110011101001",
        "1101100111101001",
        "1100110111101001",
        "1001110111101001",
        "1101110010111001",
        "1101100110111001",
        "1100110110111001",
        "1001110110111001",
        "1101100011111001",
        "1100110011111001",
        "1001110011111001",
        "1100100111111001",
        "1101110010101101",
        "1101100110101101",
        "1100110110101101",
        "1001110110101101",
        "1101100011101101",
        "1100110011101101",
        "1001110011101101",
        "1100100111101101",
        "0111110111101000",
        "0111110110111000",
        "0111110011111000",
        "0111100111111000",
        "0110110111111000",
        "0011110111111000",
        "0111110110101100",
        "0111110011101100",
        "0111100111101100",
        "0110110111101100",
        "0011110111101100",
        "0111110010111100",
        "0111100110111100",
        "0110110110111100",
        "0011110110111100",
        "0111100011111100",
        "0110110011111100",
        "0011110011111100",
        "0110100111111100",
        "0111110110101001",
        "0111110011101001",
        "0111100111101001",
        "0110110111101001",
        "0011110111101001",
        "0111110010111001",
        "0111100110111001",
        "0110110110111001",
        "0011110110111001",
        "0111100011111001",
        "0110110011111001",
        "0011110011111001",
        "0110100111111001",
        "0111110010101101",
        "0111100110101101",
        "0110110110101101",
        "0011110110101101",
        "0111100011101101",
        "0110110011101101",
        "0011110011101101",
        "0110100111101101",
        "1101011111101000",
        "1101011110111000",
        "1101011011111000",
        "1101001111111000",
        "1100011111111000",
        "1001011111111000",
        "1101011110101100",
        "1101011011101100",
        "1101001111101100",
        "1100011111101100",
        "1001011111101100",
        "1101011010111100",
        "1101001110111100",
        "1100011110111100",
        "1001011110111100",
        "1101001011111100",
        "1100011011111100",
        "1001011011111100",
        "1100001111111100",
        "1101011110101001",
        "1101011011101001",
        "1101001111101001",
        "1100011111101001",
        "1001011111101001",
        "1101011010111001",
        "1101001110111001",
        "1100011110111001",
        "1001011110111001",
        "1101001011111001",
        "1100011011111001",
        "1001011011111001",
        "1100001111111001",
        "1101011010101101",
        "1101001110101101",
        "1100011110101101",
        "1001011110101101",
        "1101001011101101",
        "1100011011101101",
        "1001011011101101",
        "1100001111101101",
        "1111111101000010",
        "1111111100010010",
        "1111111001010010",
        "1111101101010010",
        "1110111101010010",
        "1011111101010010",
        "1111111100000110",
        "1111111001000110",
        "1111101101000110",
        "1110111101000110",
        "1011111101000110",
        "1111111000010110",
        "1111101100010110",
        "1110111100010110",
        "1011111100010110",
        "1111101001010110",
        "1110111001010110",
        "1011111001010110",
        "1110101101010110",
        "1111111100000011",
        "1111111001000011",
        "1111101101000011",
        "1110111101000011",
        "1011111101000011",
        "1111111000010011",
        "1111101100010011",
        "1110111100010011",
        "1011111100010011",
        "1111101001010011",
        "1110111001010011",
        "1011111001010011",
        "1110101101010011",
        "1111111000000111",
        "1111101100000111",
        "1110111100000111",
        "1011111100000111",
        "1111101001000111",
        "1110111001000111",
        "1011111001000111",
        "1110101101000111",
        "1111110111000010",
        "1111110110010010",
        "1111110011010010",
        "1111100111010010",
        "1110110111010010",
        "1011110111010010",
        "1111110110000110",
        "1111110011000110",
        "1111100111000110",
        "1110110111000110",
        "1011110111000110",
        "1111110010010110",
        "1111100110010110",
        "1110110110010110",
        "1011110110010110",
        "1111100011010110",
        "1110110011010110",
        "1011110011010110",
        "1110100111010110",
        "1111110110000011",
        "1111110011000011",
        "1111100111000011",
        "1110110111000011",
        "1011110111000011",
        "1111110010010011",
        "1111100110010011",
        "1110110110010011",
        "1011110110010011",
        "1111100011010011",
        "1110110011010011",
        "1011110011010011",
        "1110100111010011",
        "1111110010000111",
        "1111100110000111",
        "1110110110000111",
        "1011110110000111",
        "1111100011000111",
        "1110110011000111",
        "1011110011000111",
        "1110100111000111",
        "1111011111000010",
        "1111011110010010",
        "1111011011010010",
        "1111001111010010",
        "1110011111010010",
        "1011011111010010",
        "1111011110000110",
        "1111011011000110",
        "1111001111000110",
        "1110011111000110",
        "1011011111000110",
        "1111011010010110",
        "1111001110010110",
        "1110011110010110",
        "1011011110010110",
        "1111001011010110",
        "1110011011010110",
        "1011011011010110",
        "1110001111010110",
        "1111011110000011",
        "1111011011000011",
        "1111001111000011",
        "1110011111000011",
        "1011011111000011",
        "1111011010010011",
        "1111001110010011",
        "1110011110010011",
        "1011011110010011",
        "1111001011010011",
        "1110011011010011",
        "1011011011010011",
        "1110001111010011",
        "1111011010000111",
        "1111001110000111",
        "1110011110000111",
        "1011011110000111",
        "1111001011000111",
        "1110011011000111",
        "1011011011000111",
        "1110001111000111",
        "1101111111000010",
        "1101111110010010",
        "1101111011010010",
        "1101101111010010",
        "1100111111010010",
        "1001111111010010",
        "1101111110000110",
        "1101111011000110",
        "1101101111000110",
        "1100111111000110",
        "1001111111000110",
        "1101111010010110",
        "1101101110010110",
        "1100111110010110",
        "1001111110010110",
        "1101101011010110",
        "1100111011010110",
        "1001111011010110",
        "1100101111010110",
        "1101111110000011",
        "1101111011000011",
        "1101101111000011",
        "1100111111000011",
        "1001111111000011",
        "1101111010010011",
        "1101101110010011",
        "1100111110010011",
        "1001111110010011",
        "1101101011010011",
        "1100111011010011",
        "1001111011010011",
        "1100101111010011",
        "1101111010000111",
        "1101101110000111",
        "1100111110000111",
        "1001111110000111",
        "1101101011000111",
        "1100111011000111",
        "1001111011000111",
        "1100101111000111",
        "0111111111000010",
        "0111111110010010",
        "0111111011010010",
        "0111101111010010",
        "0110111111010010",
        "0011111111010010",
        "0111111110000110",
        "0111111011000110",
        "0111101111000110",
        "0110111111000110",
        "0011111111000110",
        "0111111010010110",
        "0111101110010110",
        "0110111110010110",
        "0011111110010110",
        "0111101011010110",
        "0110111011010110",
        "0011111011010110",
        "0110101111010110",
        "0111111110000011",
        "0111111011000011",
        "0111101111000011",
        "0110111111000011",
        "0011111111000011",
        "0111111010010011",
        "0111101110010011",
        "0110111110010011",
        "0011111110010011",
        "0111101011010011",
        "0110111011010011",
        "0011111011010011",
        "0110101111010011",
        "0111111010000111",
        "0111101110000111",
        "0110111110000111",
        "0011111110000111",
        "0111101011000111",
        "0110111011000111",
        "0011111011000111",
        "0110101111000111",
        "1111110101100010",
        "1111110100110010",
        "1111110001110010",
        "1111100101110010",
        "1110110101110010",
        "1011110101110010",
        "1111110100100110",
        "1111110001100110",
        "1111100101100110",
        "1110110101100110",
        "1011110101100110",
        "1111110000110110",
        "1111100100110110",
        "1110110100110110",
        "1011110100110110",
        "1111100001110110",
        "1110110001110110",
        "1011110001110110",
        "1110100101110110",
        "1111110100100011",
        "1111110001100011",
        "1111100101100011",
        "1110110101100011",
        "1011110101100011",
        "1111110000110011",
        "1111100100110011",
        "1110110100110011",
        "1011110100110011",
        "1111100001110011",
        "1110110001110011",
        "1011110001110011",
        "1110100101110011",
        "1111110000100111",
        "1111100100100111",
        "1110110100100111",
        "1011110100100111",
        "1111100001100111",
        "1110110001100111",
        "1011110001100111",
        "1110100101100111",
        "1111011101100010",
        "1111011100110010",
        "1111011001110010",
        "1111001101110010",
        "1110011101110010",
        "1011011101110010",
        "1111011100100110",
        "1111011001100110",
        "1111001101100110",
        "1110011101100110",
        "1011011101100110",
        "1111011000110110",
        "1111001100110110",
        "1110011100110110",
        "1011011100110110",
        "1111001001110110",
        "1110011001110110",
        "1011011001110110",
        "1110001101110110",
        "1111011100100011",
        "1111011001100011",
        "1111001101100011",
        "1110011101100011",
        "1011011101100011",
        "1111011000110011",
        "1111001100110011",
        "1110011100110011",
        "1011011100110011",
        "1111001001110011",
        "1110011001110011",
        "1011011001110011",
        "1110001101110011",
        "1111011000100111",
        "1111001100100111",
        "1110011100100111",
        "1011011100100111",
        "1111001001100111",
        "1110011001100111",
        "1011011001100111",
        "1110001101100111",
        "1101111101100010",
        "1101111100110010",
        "1101111001110010",
        "1101101101110010",
        "1100111101110010",
        "1001111101110010",
        "1101111100100110",
        "1101111001100110",
        "1101101101100110",
        "1100111101100110",
        "1001111101100110",
        "1101111000110110",
        "1101101100110110",
        "1100111100110110",
        "1001111100110110",
        "1101101001110110",
        "1100111001110110",
        "1001111001110110",
        "1100101101110110",
        "1101111100100011",
        "1101111001100011",
        "1101101101100011",
        "1100111101100011",
        "1001111101100011",
        "1101111000110011",
        "1101101100110011",
        "1100111100110011",
        "1001111100110011",
        "1101101001110011",
        "1100111001110011",
        "1001111001110011",
        "1100101101110011",
        "1101111000100111",
        "1101101100100111",
        "1100111100100111",
        "1001111100100111",
        "1101101001100111",
        "1100111001100111",
        "1001111001100111",
        "1100101101100111",
        "0111111101100010",
        "0111111100110010",
        "0111111001110010",
        "0111101101110010",
        "0110111101110010",
        "0011111101110010",
        "0111111100100110",
        "0111111001100110",
        "0111101101100110",
        "0110111101100110",
        "0011111101100110",
        "0111111000110110",
        "0111101100110110",
        "0110111100110110",
        "0011111100110110",
        "0111101001110110",
        "0110111001110110",
        "0011111001110110",
        "0110101101110110",
        "0111111100100011",
        "0111111001100011",
        "0111101101100011",
        "0110111101100011",
        "0011111101100011",
        "0111111000110011",
        "0111101100110011",
        "0110111100110011",
        "0011111100110011",
        "0111101001110011",
        "0110111001110011",
        "0011111001110011",
        "0110101101110011",
        "0111111000100111",
        "0111101100100111",
        "0110111100100111",
        "0011111100100111",
        "0111101001100111",
        "0110111001100111",
        "0011111001100111",
        "0110101101100111",
        "1111010111100010",
        "1111010110110010",
        "1111010011110010",
        "1111000111110010",
        "1110010111110010",
        "1011010111110010",
        "1111010110100110",
        "1111010011100110",
        "1111000111100110",
        "1110010111100110",
        "1011010111100110",
        "1111010010110110",
        "1111000110110110",
        "1110010110110110",
        "1011010110110110",
        "1111000011110110",
        "1110010011110110",
        "1011010011110110",
        "1110000111110110",
        "1111010110100011",
        "1111010011100011",
        "1111000111100011",
        "1110010111100011",
        "1011010111100011",
        "1111010010110011",
        "1111000110110011",
        "1110010110110011",
        "1011010110110011",
        "1111000011110011",
        "1110010011110011",
        "1011010011110011",
        "1110000111110011",
        "1111010010100111",
        "1111000110100111",
        "1110010110100111",
        "1011010110100111",
        "1111000011100111",
        "1110010011100111",
        "1011010011100111",
        "1110000111100111",
        "1101110111100010",
        "1101110110110010",
        "1101110011110010",
        "1101100111110010",
        "1100110111110010",
        "1001110111110010",
        "1101110110100110",
        "1101110011100110",
        "1101100111100110",
        "1100110111100110",
        "1001110111100110",
        "1101110010110110",
        "1101100110110110",
        "1100110110110110",
        "1001110110110110",
        "1101100011110110",
        "1100110011110110",
        "1001110011110110",
        "1100100111110110",
        "1101110110100011",
        "1101110011100011",
        "1101100111100011",
        "1100110111100011",
        "1001110111100011",
        "1101110010110011",
        "1101100110110011",
        "1100110110110011",
        "1001110110110011",
        "1101100011110011",
        "1100110011110011",
        "1001110011110011",
        "1100100111110011",
        "1101110010100111",
        "1101100110100111",
        "1100110110100111",
        "1001110110100111",
        "1101100011100111",
        "1100110011100111",
        "1001110011100111",
        "1100100111100111",
        "0111110111100010",
        "0111110110110010",
        "0111110011110010",
        "0111100111110010",
        "0110110111110010",
        "0011110111110010",
        "0111110110100110",
        "0111110011100110",
        "0111100111100110",
        "0110110111100110",
        "0011110111100110",
        "0111110010110110",
        "0111100110110110",
        "0110110110110110",
        "0011110110110110",
        "0111100011110110",
        "0110110011110110",
        "0011110011110110",
        "0110100111110110",
        "0111110110100011",
        "0111110011100011",
        "0111100111100011",
        "0110110111100011",
        "0011110111100011",
        "0111110010110011",
        "0111100110110011",
        "0110110110110011",
        "0011110110110011",
        "0111100011110011",
        "0110110011110011",
        "0011110011110011",
        "0110100111110011",
        "0111110010100111",
        "0111100110100111",
        "0110110110100111",
        "0011110110100111",
        "0111100011100111",
        "0110110011100111",
        "0011110011100111",
        "0110100111100111",
        "1101011111100010",
        "1101011110110010",
        "1101011011110010",
        "1101001111110010",
        "1100011111110010",
        "1001011111110010",
        "1101011110100110",
        "1101011011100110",
        "1101001111100110",
        "1100011111100110",
        "1001011111100110",
        "1101011010110110",
        "1101001110110110",
        "1100011110110110",
        "1001011110110110",
        "1101001011110110",
        "1100011011110110",
        "1001011011110110",
        "1100001111110110",
        "1101011110100011",
        "1101011011100011",
        "1101001111100011",
        "1100011111100011",
        "1001011111100011",
        "1101011010110011",
        "1101001110110011",
        "1100011110110011",
        "1001011110110011",
        "1101001011110011",
        "1100011011110011",
        "1001011011110011",
        "1100001111110011",
        "1101011010100111",
        "1101001110100111",
        "1100011110100111",
        "1001011110100111",
        "1101001011100111",
        "1100011011100111",
        "1001011011100111",
        "1100001111100111",
        "1111110101001010",
        "1111110100011010",
        "1111110001011010",
        "1111100101011010",
        "1110110101011010",
        "1011110101011010",
        "1111110100001110",
        "1111110001001110",
        "1111100101001110",
        "1110110101001110",
        "1011110101001110",
        "1111110000011110",
        "1111100100011110",
        "1110110100011110",
        "1011110100011110",
        "1111100001011110",
        "1110110001011110",
        "1011110001011110",
        "1110100101011110",
        "1111110100001011",
        "1111110001001011",
        "1111100101001011",
        "1110110101001011",
        "1011110101001011",
        "1111110000011011",
        "1111100100011011",
        "1110110100011011",
        "1011110100011011",
        "1111100001011011",
        "1110110001011011",
        "1011110001011011",
        "1110100101011011",
        "1111110000001111",
        "1111100100001111",
        "1110110100001111",
        "1011110100001111",
        "1111100001001111",
        "1110110001001111",
        "1011110001001111",
        "1110100101001111",
        "1111011101001010",
        "1111011100011010",
        "1111011001011010",
        "1111001101011010",
        "1110011101011010",
        "1011011101011010",
        "1111011100001110",
        "1111011001001110",
        "1111001101001110",
        "1110011101001110",
        "1011011101001110",
        "1111011000011110",
        "1111001100011110",
        "1110011100011110",
        "1011011100011110",
        "1111001001011110",
        "1110011001011110",
        "1011011001011110",
        "1110001101011110",
        "1111011100001011",
        "1111011001001011",
        "1111001101001011",
        "1110011101001011",
        "1011011101001011",
        "1111011000011011",
        "1111001100011011",
        "1110011100011011",
        "1011011100011011",
        "1111001001011011",
        "1110011001011011",
        "1011011001011011",
        "1110001101011011",
        "1111011000001111",
        "1111001100001111",
        "1110011100001111",
        "1011011100001111",
        "1111001001001111",
        "1110011001001111",
        "1011011001001111",
        "1110001101001111",
        "1101111101001010",
        "1101111100011010",
        "1101111001011010",
        "1101101101011010",
        "1100111101011010",
        "1001111101011010",
        "1101111100001110",
        "1101111001001110",
        "1101101101001110",
        "1100111101001110",
        "1001111101001110",
        "1101111000011110",
        "1101101100011110",
        "1100111100011110",
        "1001111100011110",
        "1101101001011110",
        "1100111001011110",
        "1001111001011110",
        "1100101101011110",
        "1101111100001011",
        "1101111001001011",
        "1101101101001011",
        "1100111101001011",
        "1001111101001011",
        "1101111000011011",
        "1101101100011011",
        "1100111100011011",
        "1001111100011011",
        "1101101001011011",
        "1100111001011011",
        "1001111001011011",
        "1100101101011011",
        "1101111000001111",
        "1101101100001111",
        "1100111100001111",
        "1001111100001111",
        "1101101001001111",
        "1100111001001111",
        "1001111001001111",
        "1100101101001111",
        "0111111101001010",
        "0111111100011010",
        "0111111001011010",
        "0111101101011010",
        "0110111101011010",
        "0011111101011010",
        "0111111100001110",
        "0111111001001110",
        "0111101101001110",
        "0110111101001110",
        "0011111101001110",
        "0111111000011110",
        "0111101100011110",
        "0110111100011110",
        "0011111100011110",
        "0111101001011110",
        "0110111001011110",
        "0011111001011110",
        "0110101101011110",
        "0111111100001011",
        "0111111001001011",
        "0111101101001011",
        "0110111101001011",
        "0011111101001011",
        "0111111000011011",
        "0111101100011011",
        "0110111100011011",
        "0011111100011011",
        "0111101001011011",
        "0110111001011011",
        "0011111001011011",
        "0110101101011011",
        "0111111000001111",
        "0111101100001111",
        "0110111100001111",
        "0011111100001111",
        "0111101001001111",
        "0110111001001111",
        "0011111001001111",
        "0110101101001111",
        "1111010111001010",
        "1111010110011010",
        "1111010011011010",
        "1111000111011010",
        "1110010111011010",
        "1011010111011010",
        "1111010110001110",
        "1111010011001110",
        "1111000111001110",
        "1110010111001110",
        "1011010111001110",
        "1111010010011110",
        "1111000110011110",
        "1110010110011110",
        "1011010110011110",
        "1111000011011110",
        "1110010011011110",
        "1011010011011110",
        "1110000111011110",
        "1111010110001011",
        "1111010011001011",
        "1111000111001011",
        "1110010111001011",
        "1011010111001011",
        "1111010010011011",
        "1111000110011011",
        "1110010110011011",
        "1011010110011011",
        "1111000011011011",
        "1110010011011011",
        "1011010011011011",
        "1110000111011011",
        "1111010010001111",
        "1111000110001111",
        "1110010110001111",
        "1011010110001111",
        "1111000011001111",
        "1110010011001111",
        "1011010011001111",
        "1110000111001111",
        "1101110111001010",
        "1101110110011010",
        "1101110011011010",
        "1101100111011010",
        "1100110111011010",
        "1001110111011010",
        "1101110110001110",
        "1101110011001110",
        "1101100111001110",
        "1100110111001110",
        "1001110111001110",
        "1101110010011110",
        "1101100110011110",
        "1100110110011110",
        "1001110110011110",
        "1101100011011110",
        "1100110011011110",
        "1001110011011110",
        "1100100111011110",
        "1101110110001011",
        "1101110011001011",
        "1101100111001011",
        "1100110111001011",
        "1001110111001011",
        "1101110010011011",
        "1101100110011011",
        "1100110110011011",
        "1001110110011011",
        "1101100011011011",
        "1100110011011011",
        "1001110011011011",
        "1100100111011011",
        "1101110010001111",
        "1101100110001111",
        "1100110110001111",
        "1001110110001111",
        "1101100011001111",
        "1100110011001111",
        "1001110011001111",
        "1100100111001111",
        "0111110111001010",
        "0111110110011010",
        "0111110011011010",
        "0111100111011010",
        "0110110111011010",
        "0011110111011010",
        "0111110110001110",
        "0111110011001110",
        "0111100111001110",
        "0110110111001110",
        "0011110111001110",
        "0111110010011110",
        "0111100110011110",
        "0110110110011110",
        "0011110110011110",
        "0111100011011110",
        "0110110011011110",
        "0011110011011110",
        "0110100111011110",
        "0111110110001011",
        "0111110011001011",
        "0111100111001011",
        "0110110111001011",
        "0011110111001011",
        "0111110010011011",
        "0111100110011011",
        "0110110110011011",
        "0011110110011011",
        "0111100011011011",
        "0110110011011011",
        "0011110011011011",
        "0110100111011011",
        "0111110010001111",
        "0111100110001111",
        "0110110110001111",
        "0011110110001111",
        "0111100011001111",
        "0110110011001111",
        "0011110011001111",
        "0110100111001111",
        "1101011111001010",
        "1101011110011010",
        "1101011011011010",
        "1101001111011010",
        "1100011111011010",
        "1001011111011010",
        "1101011110001110",
        "1101011011001110",
        "1101001111001110",
        "1100011111001110",
        "1001011111001110",
        "1101011010011110",
        "1101001110011110",
        "1100011110011110",
        "1001011110011110",
        "1101001011011110",
        "1100011011011110",
        "1001011011011110",
        "1100001111011110",
        "1101011110001011",
        "1101011011001011",
        "1101001111001011",
        "1100011111001011",
        "1001011111001011",
        "1101011010011011",
        "1101001110011011",
        "1100011110011011",
        "1001011110011011",
        "1101001011011011",
        "1100011011011011",
        "1001011011011011",
        "1100001111011011",
        "1101011010001111",
        "1101001110001111",
        "1100011110001111",
        "1001011110001111",
        "1101001011001111",
        "1100011011001111",
        "1001011011001111",
        "1100001111001111",
    ],
}

To access the ground state energy, use the "energy" key. The "eigenvector" key provides the CI coefficients with corresponding bitstring notation of electron configuration stored with "states" of the results.

In [ ]:
fci_energy = -55.521148034704126  # the exact energy using FCI method
hivqe_energy = result["energy"]
print(
    f"|Exact Energy - HiVQE Energy|: {abs(fci_energy-hivqe_energy)*1000} mHa"
)
print(f"Sampled Number of States: {len(result["states"])}")

Output:

|Exact Energy - HiVQE Energy|: 0.077237504 mHa
Sampled Number of States: 1936

## Performance

This section shows the demonstrated benchmark calculations of HiVQE with a 24-qubit case for Li2S, a 40-qubit case for an N2 molecule, and a 44-qubit case for an FeP-NO system.

#### Dissociation potential energy surface curve for an Li2S molecule with 24 qubits

The PES curve is shown with the FCI reference and initial guess from RHF, together with the energy error from the FCI reference.

![Image showing that HiVQE produces solutions within chemical accuracy of a classical reference PES curve for the Li2S system.](/images/guides/qunova-chemistry/Li2S_PES.avif)

The calculations have been conducted with the following geometries and options.

In [ ]:
# Define Li2S geometries
Li2S_geoms = {
    "Li2S_1.51": "S        -1.239044    0.671232   -0.030374;Li       -1.506327    0.432403   -1.498949;Li       -0.899996    0.973348    1.826768;",
    "Li2S_2.40": "S        -1.741432    0.680397    0.346702;Li       -0.529307    0.488006   -1.729343;Li       -1.284307    0.989409    2.177209;",
    "Li2S_3.80": "S        -2.707255    0.674298    0.909161;Li        0.079218    0.552012   -1.671656;Li       -0.927010    0.931502    1.557063;",
}

# Configure some options for the job.
molecule_options = {
    "basis": "sto3g",
}
hivqe_options = {
    "shots": 100,
    "max_iter": 20,
}

results = []
for geom in ["Li2S_1.51", "Li2S_2.40", "Li2S_3.80"]:
    # Run HiVQE
    job = function.run(
        geometry=Li2S_geoms[geom],
        backend_name=backend_name,  # e.g. "ibm_fez"
        instance=instance,  # e.g. "ibm-q/open/main"
        max_states=2000,
        max_expansion_states=10,
        molecule_options=molecule_options,
        hivqe_options=hivqe_options,
    )
    results.append(job.result())

The red dots represent the HiVQE calculation results for six different geometries, and three geometries corresponding to 1.51, 2.40, and 3.80 Angstrom are provided as input in the above cell.

#### Dissociation PES curve for an N2 molecule with 40 qubits

The nitrogen molecule has been identified as a multireference system with large correlation energy contributions beyond the Hartree-Fock state. We conducted a benchmark calculation for the N2 molecule with cc-pvdz basis, (20o,14e) using the homo-lumo active orbital selection. The complete active space (CAS) number to represent this problem is 6,009,350,400. It is not possible to obtain the eigenvalue problem solution (for energy and electronic structure) with this number of states using a powerful desktop (16cpu/64GB). With HiVQE, users can efficiently search the subspace of CAS states to find chemically accurate results while saving computation resources significantly. The following plots show the PES curve of 40 qubits HiVQE calculation of the N2 molecule dissociation.

![Image showing that HiVQE produces solutions within chemical accuracy of a classical reference PES curve for the N2 system.](/images/guides/qunova-chemistry/N2_PES_40qubits.avif)

#### Dissociation PES curve for five-coordinated iron(II)-porphyrin with an NO system with 44 qubits

Another interesting chemical system is an iron(II)-porphyrin (FeP) complex with a coordinated nitric oxide (NO) ligand, which represents a biologically relevant metalloporphyrin system that plays crucial roles in various physiological processes. In this example, HiVQE has been utilized to estimate the accurate potential energy surface curve of the intermolecular interaction between FeP and NO (ground state energy for differently separated geometries). The combined system has 450 orbitals and 202 electrons (450o,202e) with 6-31g(d) basis in total. The homo-lumo active orbital selection was utilized to calculate the smaller case from the real case with (22o,22e). From the following benchmark results, we were able to achieve the chemical accuracy (>&nbsp;1.6 mHa) with a state-of-the-art classical computer chemistry calculation of CASCI(DMRG) (22o,22e) reference.

![Image showing that HiVQE produces solutions within chemical accuracy of a classical reference PES curve for the FeP-NO system.](/images/guides/qunova-chemistry/fepno_44qubits.avif)

## Benchmarks

- Exact matrix size is the number of determinants for exact solution, such as FCI and CASCI.
- HiVQE calculation samples and calculates the subspace of it (as in, HiVQE matrix size).
- Total time includes QPU runtime and Qiskit Function runs with CPU.
- Accuracy is estimated from the energy difference from exact solution.

|   Chemical system  | Number of qubits | Exact matrix size |  HiVQE matrix size  | E(diff) from exact (mHa) | Number of iteration | Total time    | QPU runtime usage |
| ------------------ | ---------------- | ----------------- | ------------------- | ------------------------ | ------------------- | ------------- | ----------------- |
|  $NH_3$ (8o,10e)   |  16              |  3136             |  1936               |  0.08                    |  6                  | 37 s          | 34 s              |
|  $Li_2S$ (10o,10e) |  20              |  63504            |  3969               |  0.60                    |  5                  | 250 s         | 50 s              |
|  $NH_3$ (15o,10e)  |  30              |  9018009          |  49729              |  0.90                    |  5                  | 354 s         | 54 s              |
|  $N_2$ (16o,14e)   |  32              |  130873600        |  1798281            |  1.10                    |  9                  | 6531 s        | 121 s             |
|  $3H_2O$ (18o,24e) |  36              |  344622096        |  399424             |  0.90                    |  24                 | 5174 s        | 130 s             |
|  $N_2$ (20o,14e)   |  40              |  6009350400       |  9012004            |  1.20                    |  21                 | 46547 s       | 258 s             |

## Fetch error messages

If your workload status is `ERROR`, use `job.result()` to fetch the error message to help debug as follows:

In [ ]:
job = function.run(
    geometry=geometry,
    backend_name=backend.name,
    max_states=2000,
    max_expansion_states=15,
    molecule_options=molecule_options,
    hivqe_options=hivqe_options,
)

result = job.result()
if job.status() == "ERROR":
    print("Status:", result["status"])
    print("Message:", result["message"])

## Get support

You can send an email to [qiskit.support@qunovacomputing.com](mailto:qiskit.support@qunovacomputing.com) for help with this function.

If you want help with troubleshooting a specific error, please provide the Function job ID of the job that encountered the error.

## Next steps

<Admonition type="tip" title="Recommendations">
- Read [the associated research paper](https://qunova-my.sharepoint.com/:b:/g/personal/pellow_r_qunovacomputing_com/EcDyE4yQD_NAppzyUvhnlrABIMlqY3JhrLS41hkBPk22uQ?e=2dx3JT).
- Request access to the function by filling in [this form](https://forms.office.com/r/zN3hvMTqJ1).
- Try the [Compute dissociation PES curve for FeP-NO with HiVQE](https://learning.quantum.ibm.com/tutorial/qunova-hivqe) tutorial.
</Admonition>